In [3]:
from helper_utils import load_chroma, word_wrap
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import const
import json
import requests

In [4]:
embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = load_chroma(filename='/teamspace/studios/this_studio/ProjectGenAI/data/BOE-020_Codigo_de_Trafico_y_Seguridad_Vial.pdf', collection_name='boe_normativa_trafico', embedding_function=embedding_function)
chroma_collection.count()

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


11566

In [30]:
original_query = "Si no me paro a socorrer en caso de accidente, que me puede pasar?"

## Basic retrieval

In [31]:
results = chroma_collection.query(query_texts=[original_query], n_results=5)
retrieved_documents = results['documents'][0]

## Query expansion

### Expansion generated

In [19]:
# Function to expand users query witha hypothetical answer
def augment_generated_query(query):
    
    # Authorization header for the API
    headers = {"Authorization": f"Bearer {const.API_KEYalberto}"}
    # URL for the API
    url = "https://api.edenai.run/v2/text/chat"
    
    payload = {
    "providers": "openai",
    "text": query,
    "chatbot_global_action": "Eres un asistente experto en la normativa de tráfico de la Dirección General de Tráfico (DGT), el organismo \
                              un organismo autónomo del Gobierno de España dependiente del Ministerio del Interior responsable de la ejecución de la \
                              política vial en las vías de titularidad estatal de España."
                "Sugiere una respuesta de ejemplo a la pregunta realizada, algo como lo que podría encontrarse en la normativa de la DGT",
    "previous_history": [],
    "temperature": 0.0,
    "max_tokens": 150,
    }

    # Send the request to the API
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)
    augmented_query = result['openai']['generated_text']
    # augmented_queries = augmented_queries.split('\n')
    
    return augmented_query

hypothetical_answer = augment_generated_query(original_query)
print(hypothetical_answer)
joint_query = f"{original_query} {hypothetical_answer}"

results = chroma_collection.query(query_texts=joint_query, n_results=5, include=['documents', 'embeddings'])

# Final documents passed to the chatbot function
retrieved_documents = results['documents'][0]

Según la normativa de tráfico de la Dirección General de Tráfico (DGT), como peatón no puedes circular por las autopistas, autovías, vías interurbanas y túneles, salvo que exista una señalización específica que lo permita. Es importante recordar que los peatones deben circular por las aceras, pasos de peatones y zonas habilitadas para ellos, respetando siempre las normas de tráfico y priorizando su seguridad.


### Expansion multiple queries

In [28]:
# Function that augments a query with multiple questions related to the original query
def augment_multiple_query(query):
    
    # Authorization header for the API
    headers = {"Authorization": f"Bearer {const.API_KEYalberto}"}
    # URL for the API
    url = "https://api.edenai.run/v2/text/chat"
    
    payload = {
    "providers": "openai",
    "text": query,
    "chatbot_global_action": "Eres un asistente experto en la normativa de tráfico de la Dirección General de Tráfico (DGT), el organismo \
                              un organismo autónomo del Gobierno de España dependiente del Ministerio del Interior responsable de la ejecución de la \
                              política vial en las vías de titularidad estatal de España."
                    "Sugiere hasta cinco preguntas adicionales relacionadas para ayudarles a encontrar la información que necesitan para la pregunta proporcionada."
                    "Sugiere solo preguntas cortas sin oraciones compuestas. Sugiere una variedad de preguntas que cubran diferentes aspectos del tema."
                    "Asegúrate de que sean preguntas completas y que estén relacionadas con la pregunta original."
                    "La salida debe ser una pregunta por línea. No numeres las preguntas.",
    "previous_history": [],
    "temperature": 0.0,
    "max_tokens": 150,
    }

    # Send the request to the API
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)
    augmented_queries = result['openai']['generated_text']
    augmented_queries = augmented_queries.split('\n')
    
    return augmented_queries

augmented_queries = augment_multiple_query(original_query)

for query in augmented_queries:
    print(query)

queries = [original_query] + augmented_queries
results = chroma_collection.query(query_texts=queries, n_results=3, include=['documents', 'embeddings'])
retrieved_documents = results['documents']

# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

# Final documents passed to the chatbot function
retrieved_documents = unique_documents

'''
for i, documents in enumerate(retrieved_documents):
    print(f"Query: {queries[i]}")
    print('')
    print("Results:")
    for doc in documents:
        print(word_wrap(doc))
        print('')
    print('-'*100)
'''

¿Cuál es el procedimiento a seguir al tener un accidente de tráfico? 
¿Qué información debo recopilar en caso de presenciar un accidente de tráfico? 
¿Cuál es la forma correcta de ayudar a las víctimas de un accidente de tráfico? 
¿Qué debo hacer si soy testigo de un accidente de tráfico pero no estoy involucrado en él? 
¿Cuál es la manera adecuada de alertar a los servicios de emergencia en caso de presenciar un accidente de tráfico?


'\nfor i, documents in enumerate(retrieved_documents):\n    print(f"Query: {queries[i]}")\n    print(\'\')\n    print("Results:")\n    for doc in documents:\n        print(word_wrap(doc))\n        print(\'\')\n    print(\'-\'*100)\n'

In [23]:
def chatbot(query,retrieved_documents):

    information = "\n\n".join(retrieved_documents)
    final_query = f"Pregunta: {query} \n Información: {information}"

    # Authorization header for the API
    headers = {"Authorization": f"Bearer {const.API_KEYalberto}"}
    # URL for the API
    url = "https://api.edenai.run/v2/text/chat"

    payload = {
    "providers": "openai",
    "text": final_query,
    "chatbot_global_action": "Eres un asistente experto en la normativa de tráfico de la Dirección General de Tráfico (DGT), el organismo \
                              un organismo autónomo del Gobierno de España dependiente del Ministerio del Interior responsable de la ejecución de la \
                              política vial en las vías de titularidad estatal de España."
            "Tu tarea es ayudar a los usuarios a comprender y responder preguntas sobre el contenido de la normativa de tráfico de la DGT. "
            "Te proporcionaré la pregunta del usuario y la información relevante extraída de la normativa. "
            "Responde a la pregunta del usuario utilizando ÚNICAMENTE la información proporcionada. ",
    "previous_history": [],
    "temperature": 0.0,
    "max_tokens": 150,
    }

    # Send the request to the API
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)
    final_response = result['openai']['generated_text']

    return final_response

In [32]:
answer = chatbot(original_query,retrieved_documents)
print(answer)

Respuesta: Según la normativa de tráfico de la DGT, si no te paras a socorrer en caso de accidente de circulación, podrías recibir una multa de hasta 30.000 euros y la retirada de puntos del carnet de conducir. Además, en caso de no prestar auxilio a los heridos, podrías ser sancionado con penas de prisión de hasta cuatro años. Es importante recordar que es un deber prestar ayuda en caso de accidente de tráfico.
